In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("/home/jiaruil5/personality/llm_personality/psychgenerator_opensource/data/big5_training_data.csv", header = 0, index_col = 0)
df

,message,ope_z,con_z,ext_z,agr_z,neu_z
message_id,,,,,,
tweet_143508,""";;tengo mi penchi fierron up in the air & its...",-0.300409,0.459249,0.294330,0.452765,-0.894041
tweet_425070,perhaps he loved everyone but me. nbut that ma...,0.896169,0.493197,-0.414594,0.337368,0.393312
tweet_502775,"if you have 3 coffee cups in your car, you hav...",0.213276,0.176036,0.328285,0.274713,-0.211072
tweet_600640,i'm too sleepy to start work in an hour,-0.711772,1.284832,-1.013234,0.611918,0.545335
blog_79108,i worked for a recruiter a little bit after i ...,1.837614,0.042504,-0.460709,0.061335,1.808086
...,...,...,...,...,...,...
blog_46845,hey what do u think of 'unwell' by matchbox 20...,0.076771,-0.211666,0.577826,0.861935,-0.708217
tweet_611609,my life is dedicated to school right now ready...,-1.700926,1.092124,1.216876,1.034062,-0.978274
blog_107867,bush and cheney continue to assert that there ...,2.278704,0.275908,0.107220,-1.885664,-0.970312


In [2]:
df = df.sample(frac=1, random_state=42)

In [3]:
df[['ope_z', 'con_z', 'ext_z', 'agr_z', 'neu_z']].describe()

,ope_z,con_z,ext_z,agr_z,neu_z
count,8.463040e+05,8.463040e+05,8.463040e+05,8.463040e+05,8.463040e+05
mean,-4.728533e-17,-2.757192e-17,-1.863875e-17,1.193048e-17,1.726183e-17
std,9.999994e-01,9.999994e-01,9.999994e-01,9.999994e-01,9.999994e-01
min,-1.791328e+01,-1.606235e+01,-1.647491e+01,-2.421485e+01,-9.958985e+00
25%,-5.990037e-01,-5.480083e-01,-5.593109e-01,-4.808781e-01,-5.858125e-01
50%,-6.985836e-02,5.923059e-02,4.827777e-03,1.119149e-01,-1.141641e-01
75%,5.346785e-01,5.938862e-01,5.516231e-01,5.968365e-01,4.446708e-01
max,1.507327e+01,2.519352e+01,2.000737e+01,1.254866e+01,3.199032e+01


In [4]:
def label_by_percentile(value, p50):
    if value <= p50:
        return 0
    else:
        return 1

def dataset_filter(df):
    z_columns = [col for col in df.columns if col.endswith('_z')]
    
    for col in z_columns:
        p50 = df[col].quantile(0.5)
        print(f"Column: {col}, the middle threshold: {p50}")
        df[f'{col}_label'] = df[col].apply(lambda x: label_by_percentile(x, p50))
    
    return df

filtered_df = dataset_filter(df)

Column: ope_z, the middle threshold: -0.0698583615952959
Column: con_z, the middle threshold: 0.0592305890449004
Column: ext_z, the middle threshold: 0.00482777746889995
Column: agr_z, the middle threshold: 0.11191485748402755
Column: neu_z, the middle threshold: -0.1141640793829031


In [5]:
print(filtered_df.isnull().sum())

message        0
ope_z          0
con_z          0
ext_z          0
agr_z          0
neu_z          0
ope_z_label    0
con_z_label    0
ext_z_label    0
agr_z_label    0
neu_z_label    0
dtype: int64


In [6]:
filter_df = filtered_df.drop(columns=["ope_z", "con_z", "ext_z", "agr_z", "neu_z"])
filter_df

,message,ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
message_id,,,,,,
tweet_79485,afternoon beer and football. love bank hol wee...,0,1,1,1,0
blog_285431,i'm not much of a curser.i have a pretty exten...,1,0,0,0,1
tweet_592581,spent the entire day with my momma. and it was...,1,1,1,1,1
blog_243831,"i layed in bed last night, restlessly tossing ...",1,0,0,0,1
blog_232829,in the a.m.this has to be the sixteenth effort...,1,0,1,1,1
...,...,...,...,...,...,...
tweet_156286,playing arkham asylum. i am the bat.,0,0,0,0,0
tweet_176785,the last 15 seconds i mean 10 minutes of this ...,0,1,0,0,1
blog_372010,"as you may have noticed, my post titles have b...",1,0,0,0,0


In [9]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(filter_df, test_size=0.05, random_state=42)

In [10]:
train_df.to_csv("/data/user_data/wenkail/llm_personality/data/big5_data_generator_train.csv", index=True)
test_df.to_csv("/data/user_data/wenkail/llm_personality/data/big5_data_generator_test.csv", index=True)

In [11]:
train_df

,message,ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
message_id,,,,,,
tweet_284066,i can't talk so ima dm you,0,1,0,0,0
tweet_157848,dylan and i are texting the most crude texts t...,1,1,1,1,1
tweet_102699,this weekend we wildin out to jowell y randy ??,1,1,1,1,0
tweet_539796,this is my zen. music blaring in my headphones...,1,0,0,1,1
blog_383291,wala me masabi kung hindi ang skit ng buo kong...,0,1,0,1,0
...,...,...,...,...,...,...
blog_591894,i think the presentation could have used some ...,1,1,0,0,0
blog_469118,i'll think of you and probably laugh,1,0,0,1,1
blog_385508,"so, i said the other day that i had seen steve...",0,1,1,0,1


In [12]:
test_df

,message,ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
message_id,,,,,,
tweet_207056,my phones acting a little slow.. then i rememb...,1,0,0,0,1
blog_176919,well today was the day we got our mid-term mar...,0,1,0,1,0
blog_686715,being sick sucks. merry christmas eve to all.,0,0,0,0,1
blog_270817,"life's too short.... don't know what did it, ...",0,1,0,0,1
tweet_157250,poison girl is stuck in my head urgh,1,0,0,0,1
...,...,...,...,...,...,...
tweet_637400,i need my dick sucked on,0,0,0,0,1
blog_455581,"dear..i am sorry , i shld understand ur feelin...",0,0,1,0,1
blog_117094,i've always been a collector.i've collected st...,1,0,0,1,0
